# 驱动设备

In [1]:
import ZW_AWG1000_dds8
import numpy as np
import time
import csv
import os
from matplotlib import pyplot as plt
import datetime

In [2]:
handle_data = ZW_AWG1000_dds8.AWG1000()
handle_cmd = ZW_AWG1000_dds8.AWG1000()

In [3]:
handle_cmd.connect("192.168.1.100", 9001)
handle_data.connect("192.168.1.100", 9002)

# 设置输出模式和触发模式

In [15]:
handle_cmd.set_data_source(1, 50, 0)  # 设置数据源
#参数1 设置通道 只能设置1 3 5 7 设置1 生效通道1和2，12 34 56 78 通道的模式绑定
#参数1 输出频率 ，单位：MHz
#参数2 ： 0(默认值):波形需要下载，通过trig播放，（此模式下参数1无意义）。
#         1：厂商内部使用（此模式下参数1有意义）
#         2：需trig,反复输出下载的波形。（此模式下参数1无意义）
#         3: 表示根据用户DDS参数自主生成波形模式（此模式下参数1无意义）
#结束

True

In [7]:
#设置8个mark用作触发源
t = 1e6 #单位：ns 设置mark用作trig是trig的周期
#ch：设置通道；
# mark模式1：mark用作trig,0:mark和输出对齐；
# pulse_cnt固定等于t/20表示内部周期信号反转的计数器值用来确定mark输出信号的周期
handle_cmd.set_marktrig(ch=1,mark_mode=1,pulse_cnt=int(t/20))
time.sleep(0.1)
handle_cmd.set_marktrig(ch=2,mark_mode=1,pulse_cnt=int(t/20))
time.sleep(0.1)
handle_cmd.set_marktrig(ch=3,mark_mode=1,pulse_cnt=int(t/20))
time.sleep(0.1)
handle_cmd.set_marktrig(ch=4,mark_mode=1,pulse_cnt=int(t/20))
time.sleep(0.1)
handle_cmd.set_marktrig(ch=5,mark_mode=1,pulse_cnt=int(t/20))
time.sleep(0.1)
handle_cmd.set_marktrig(ch=6,mark_mode=1,pulse_cnt=int(t/20))
time.sleep(0.1)
handle_cmd.set_marktrig(ch=7,mark_mode=1,pulse_cnt=int(t/20))
time.sleep(0.1)
handle_cmd.set_marktrig(ch=8,mark_mode=1,pulse_cnt=int(t/20))


True

In [8]:
#关闭mark触发源
handle_cmd.set_marktrig(trig_en = 0)


True

In [9]:
#开启mark触发源
handle_cmd.set_marktrig(trig_en = 1)

True

In [10]:
# 设置通道触发来源
#参数1：设置要控制的输出通道
#参数2：设置要控制的输出通道的trig信号来源，0表示后面板的trig——in ；1-8表示前面板t1-t8
#参数3：收到trig后delay n个416ps开始输出
#参数4：mark信号延迟多久输出，mark信号只有7ns脉宽
for i in range(1,9,1):
    handle_cmd.set_trigger_param(i, 1, 0, 0)  # 设置通道1-8 触发为T1


# python生成sin波 ：amp[0,1] ,freq(Hz) ,phase ,脉宽（ns）

In [ ]:
for i in range(1,9,1):
    handle_cmd.set_output_mode(i, "AC")  # 通道1设置AC模式输出

In [10]:
for i in range(1,9,1):
    handle_cmd.set_trigger_param(i, 0, 0, 0)  # 通道1设置触发参数

In [11]:
handle_cmd.close_channel([1,2,3,4,5,6,7,8])
from matplotlib import pyplot as plt
wave_data = handle_cmd.sin_wave(1,300e+6,0,800e-9) 
# plt.plot(wave_data)
for i in range(1,9):
    status = handle_data.send_waveform_data(wave_data,i)
    print(f'通道{i}数据下载成功') if status else print(f'通道{i}数据下载失败')
    time.sleep(0.1) 
handle_cmd.open_channel([1,2,3,4,5,6,7,8])  # 打开通道    

通道1数据下载成功
通道2数据下载成功
通道3数据下载成功
通道4数据下载成功
通道5数据下载成功
通道6数据下载成功
通道7数据下载成功
通道8数据下载成功


True

In [13]:
handle_cmd.close_channel([1,2,3,4,5,6,7,8])  

True

# python生成DC模式方波脉冲 ：Vset（V） ,脉宽（ns）,通道（有校准系数）

In [14]:
for i in range(1,9,1):
    handle_cmd.set_output_mode(i, "DC")  # 通道1设置DC模式输出

In [15]:
for i in range(1,9,1):
    handle_cmd.set_trigger_param(i, 1, 0, 0)  # 通道1设置触发参数

In [ ]:
handle_cmd.close_channel([1,2,3,4,5,6,7,8])  # 关闭通道
from matplotlib import pyplot as plt
# plt.plot(wave_data)
for i in range(1,9):
    wave_data = handle_cmd.pulsewave_gen(2, 600e-6,i)
    status = handle_data.send_waveform_data(wave_data,i)
    print(f'通道{i}数据下载成功') if status else print(f'通道{i}数据下载失败')
    time.sleep(0.1) 
handle_cmd.open_channel([1,2,3,4,5,6,7,8])   # 打开通道

In [ ]:
handle_cmd.close_channel([1,2,3,4,5,6,7,8])

# 波形序列方式，用户自主生成波形 

In [ ]:
handle_cmd.set_data_source(1, 50, 3)  # 设置模式为自主波形模式

In [20]:
for i in range(1,9,1):
    handle_cmd.set_output_mode(i, "AC")  # 通道1设置AC模式输出

In [21]:
handle_cmd.close_channel([1,2,3,4,5,6,7,8])  # 关闭通道

True

In [22]:
#单通道输出单波形，n个序列
trigdelay = [100]
paramx1 = []
paramx2 = []
paramx3 = []
paramx4 = []
paramx5 = []
paramx6 = []
paramx7 = []
paramx8 = []
# paramx1.append([freq, length, amp, phase_offest, win_type, win_len])
paramx1.append([20*1e6,1000,60000,90,1,400])
paramx1.append([30*1e6,500,0,90,0,200])
paramx1.append([50*1e6,500,50000,90,1,200])
paramx1.append([70*1e6,500,45000,90,1,200])
paramx1.append([90*1e6,500,40000,90,1,200])
paramx1.append([110*1e6,500,35000,90,1,200])
paramx1.append([130*1e6,500,0,90,1,200])
paramx1.append([150*1e6,500,30000,90,1,200])
paramx1.append([170*1e6,500,25000,90,1,200])
paramx1.append([190*1e6,500,60000,90,1,200])
paramx8 = []
paramx8.append(paramx1)
for i in range(1,9):
    ch= i
    handle_cmd.DefWaveGen(ch, trigdelay, paramx8)
time.sleep(0.1) 

In [17]:
#%%
handle_cmd.close_channel([1,2,3,4,5,6,7,8])
for i in range(1,9):
    ch= i
    trigdelay = [0]
    fs = 2.4e9
    wave_len =0 #连续输出，方便频谱上观看
    amp = 8000 
    phase = 0
    win_type = 0 #不加窗
    win_len = 0
    n = 8
    seqword_x8 = [[[0]]*1 for i in range(n)]
    freq = 90e6
    m = 0
    # freq,len,amp,phase,win_type,win_len
    for i in range(0,8,1):
        seqword_x8[i][0]=[int(freq + i*2e6),wave_len,amp,phase,win_type,win_len]
    handle_cmd.DefWaveGen(ch, trigdelay, seqword_x8)


In [ ]:
#90-104 step2MHz 8个频率波形叠加 连续波序列 
handle_cmd.close_channel([1,2,3,4,5,6,7,8])  # 关闭通道
ch= 2
trigdelay = [0]
fs = 2.4e9
wave_len =0 #连续输出，方便频谱上观看
amp = 8000 
phase = 0
win_type = 0 #不加窗
win_len = 0
n = 8
seqword_x8 = [[[0]]*1 for i in range(n)]
freq = 90e6
m = 0
# freq,len,amp,phase,win_type,win_len
for i in range(0,8,1):
    seqword_x8[i][0]=[int(freq + i*2e6),wave_len,amp,phase,win_type,win_len]
handle_cmd.DefWaveGen(ch, trigdelay, seqword_x8)
handle_cmd.open_channel([1,2,3,4,5,6,7,8])   # 打开通道

In [ ]:
#90-104 step2mhz频率叠加 开4us关4us波形序列
handle_cmd.open_channel([1,2,3,4,5,6,7,8])   # 打开通道
ch= 1
trigdelay = [0]
fs = 2.4e9
pulse_width = 4000e-9
wave_len = int(pulse_width/(1/fs))#脉冲
# wave_len =0 #连续输出，方便频谱上观看
amp = 8000 
phase = 0
win_type = 0
# win_len = int(wave_len / 2)
win_len = 0
seqword = []
n = 8
seqword_x8 = [[[0]]*n for i in range(n)]
freq = 10e6
m = 0
# freq,len,amp,phase,win_type,win_len
for f in range(0,n,1):
    if f % 2 :
        amp = 0
    else :
        amp = 8000
    for i in range(0,8,1):
        seqword_x8[i][f]=[int(freq + i*2e6),wave_len,amp,phase,win_type,win_len]

handle_cmd.DefWaveGen(ch, trigdelay, seqword_x8)

In [ ]:
#90mhz频率单dds 每隔1us 幅度降低10% 重复3次
ch= 1
trigdelay = [0]
fs = 2.4e9
pulse_width = 40e-9
wave_len = int(pulse_width/(1/fs))
amp = 65535 
phase = 0
win_type = 0
win_len = 0
seqword = []
n = 30 #每个dds内的波形个数
m = 1 #dds 叠加数量 1-8
seqword_x8 = [[[0]]*n for i in range(m)]
freq = 500e6
m = 1 #dds 叠加数量 1-8
# freq,len,amp,phase,win_type,win_len
for i in range(0,n,1):
    seqword_x8[0][i]=[freq,wave_len,int(amp*(1 - 0.1*(i % 10))),phase,win_type,win_len]

handle_cmd.DefWaveGen(ch, trigdelay, seqword_x8)

In [ ]:
#8个波形各自输出完整波形的一段，再拼接成完整波形，验证相位连续性
import random
a = [1,2,3,4,5,6,7,8]
random.shuffle(a)
ch= 1
trigdelay = [0]
fs = 2.4e9
pulse_width = 200e-9
wave_len = int(pulse_width/(1/fs))
amp = 50000 
phase = 0
win_type = 0
# win_len = int(wave_len / 2)
win_len = 0
seqword = []
n = 8
m = 8
seqword_x8 = [[[0]]*n for i in range(8)]
freq = 10e6
m = 0
# freq,len,amp,phase,win_type,win_len
for f in range(0,n,1):
    for i in range(0,8,1):
        if a[i] == (f % 8) + 1 :
            amp = 50000
        else :
            amp = 0 
        seqword_x8[i][f]=[int(freq),wave_len,amp,phase,win_type,win_len]            
handle_cmd.DefWaveGen(ch, trigdelay, seqword_x8)


In [ ]:
handle_cmd.open_channel([1,2,3,4,5,6,7,8])   # 打开通道

# 其他功能

In [ ]:
handle_cmd.change_dev_ip("eth","192.168.110.100")   #网口修改IP(eth：网口，uart：串口)

In [ ]:
handle_cmd.change_dev_ip("uart","192.168.110.100")   #串口修改IP(eth：网口，uart：串口)

In [33]:
handle_cmd.get_temperature() #读取设备温度

[63.893951416015625, 66.91796875, 74.70059204101562, 69.46612548828125]

In [ ]:
for i in range(1,9,1):                            #DC模式默认电压设置（初始化0V）
    handle_cmd.set_default_vbias(i,1) 

In [ ]:
for i in range(1,9,1):
    handle_cmd.set_rf_atten(i, 0)  # 通道设置输出功率衰减

In [ ]:
handle_cmd.set_refclk("ext_ref", 10)  # 设置内外参考

In [ ]:
handle_cmd.set_refclk("int_ref", 100)  # 设置内外参考

In [3]:
# import serial 

handle_cmd.open_uart("COM3")  # 打开串口3,备后面获取IP

handle_cmd.get_dev_ip()       # 

'192.168.1.104'

In [23]:
handle_cmd.set_fan_speed(4699) #设置风扇速度

True

In [ ]:
handle_cmd.open_uart("")